In [140]:
from tensorflow.keras.layers import Conv1D, Conv2D, Conv3D
import numpy as np

In [ ]:
n_filters = 10
padding = 'valid'  # No padding to the input
#padding = 'same'    # Apply padding to the input

k=40
s=30

x_1d = np.random.rand(15, 120, 3)
k_1d = (k,)
s_1d = (s,)

x_2d = np.random.rand(15, 120, 120, 3)
k_2d = (k,k)
s_2d = (s,s)

x_3d = np.random.rand(15, 120, 120, 120, 3)
k_3d = (k,k,k)
s_3d = (s,s,s)

In [142]:
def padding_computing(width, kernel, stride):
    ss = range(0,width-kernel,stride)[-1]
    #print(f"first ss: {ss}")
    while ss+kernel < width:
        ss = ss+stride
    #print(f"last ss: {ss}")
    return ((ss+kernel)-width)

# Convolution 1D

In [143]:
x = x_1d
kernel_shape = k_1d
stride_shape = s_1d

# Understaning the output shape of a Conv layer
n_dims = len(x.shape[1:-1])
input_shape = tuple(x.shape[1:])
padding_shape = tuple(np.zeros((n_dims,),dtype=np.int32))
if padding.lower() != 'valid'.lower():
    padding_shape = tuple([padding_computing(input_shape[:-1][i], kernel_shape[i], stride_shape[i]) for i in range(0,len(stride_shape))])
expected_output_shape = tuple([int((d-kernel_shape[i]+2*padding_shape[i])/stride_shape[i])+1 for i, d in enumerate(input_shape[:-1])]) + (n_filters,)
print(f'Expected output shape (Conv 1D): {expected_output_shape}')

y = Conv1D(
    filters=n_filters,
    kernel_size=kernel_shape,
    strides=stride_shape,
    padding=padding,
    name=f"test_conv_1d_layer"
)(x)
print(f'Conv1D output shape: {tuple(y.shape[1:])}')

Expected output shape (Conv 1D): (3, 10)
Conv1D output shape: (3, 10)


# Convolution 2D

In [144]:
x = x_2d
kernel_shape = k_2d
stride_shape = s_2d

# Understaning the output shape of a Conv layer
n_dims = len(x.shape[1:-1])
input_shape = tuple(x.shape[1:])
padding_shape = tuple(np.zeros((n_dims,),dtype=np.int32))
if padding.lower() != 'valid'.lower():
    padding_shape = tuple([padding_computing(input_shape[:-1][i], kernel_shape[i], stride_shape[i]) for i in range(0,len(stride_shape))])
expected_output_shape = tuple([int((d-kernel_shape[i]+2*padding_shape[i])/stride_shape[i])+1 for i, d in enumerate(input_shape[:-1])]) + (n_filters,)
print(f'Expected output shape (Conv 2D): {expected_output_shape}')

y = Conv2D(
    filters=n_filters,
    kernel_size=kernel_shape,
    strides=stride_shape,
    padding=padding,
    name=f"test_conv_2d_layer"
)(x)
print(f'Conv2D output shape: {tuple(y.shape[1:])}')

Expected output shape (Conv 2D): (3, 3, 10)
Conv2D output shape: (3, 3, 10)


# Convolution 3D

In [145]:
x = x_3d
kernel_shape = k_3d
stride_shape = s_3d

# Understaning the output shape of a Conv layer
n_dims = len(x.shape[1:-1])
input_shape = tuple(x.shape[1:])
padding_shape = tuple(np.zeros((n_dims,),dtype=np.int32))
if padding.lower() != 'valid'.lower():
    padding_shape = tuple([padding_computing(input_shape[:-1][i], kernel_shape[i], stride_shape[i]) for i in range(0,len(stride_shape))])
expected_output_shape = tuple([int((d-kernel_shape[i]+2*padding_shape[i])/stride_shape[i])+1 for i, d in enumerate(input_shape[:-1])]) + (n_filters,)
print(f'Expected output shape (Conv 3D): {expected_output_shape}')

y = Conv3D(
    filters=n_filters,
    kernel_size=kernel_shape,
    strides=stride_shape,
    padding=padding,
    name=f"test_conv_3d_layer"
)(x)
print(f'Conv3D output shape: {tuple(y.shape[1:])}')

Expected output shape (Conv 3D): (3, 3, 3, 10)
Conv3D output shape: (3, 3, 3, 10)
